In [1]:
import scanpy as sc
import pandas as pd
import decoupler as dc

In [ ]:
#去除其他染色体上的基因和一些样本以及细胞类型
adata = sc.read_h5ad('/CIMA/scRNA_Data/NatualCohort_All_Annotation_Final_reUMAP.h5ad')
tss_bed = pd.read_csv('/CIMA/Make_Gene_TSS/tss.bed',sep = ' ')
adata = adata[:,adata.var_names.isin(tss_bed['gene_id'])]
sample_with_genomics = pd.read_table('../Data/413sample.txt')
#去除没有wgs信息的样本
adata = adata[adata.obs['sample'].isin(sample_with_genomics['FID']),:]

In [7]:
adata.obs["celltype_l1"].unique()

['CD8_T', 'Myeloid', 'CD4_T', 'NK&ILC', 'B', 'unconvensional_T']
Categories (6, object): ['B', 'CD4_T', 'CD8_T', 'Myeloid', 'NK&ILC', 'unconvensional_T']

In [66]:
def preprocess_RNA_data(adata_use,celltype):
    print(f"正在处理 {celltype}", flush=True)
    #算伪Bulk
    pdata = dc.get_pseudobulk(
        adata_use,
        sample_col='sample',
        groups_col=None,
        mode='mean',
        min_cells=10,
        min_counts=0,
        min_prop=0,
        min_smpls=0)
        
    #生成伪bulk矩阵
    pseudo_matrix = pd.DataFrame(pdata.X)
    pseudo_matrix.columns = pdata.var_names
    pseudo_matrix.index = pdata.obs.index

    #只选取在90%的样本中都表达的特征
    non_zero_ratio = (pseudo_matrix != 0).mean()
    columns_to_keep = non_zero_ratio[non_zero_ratio >= 0.9].index
    pseudo_matrix = pseudo_matrix[columns_to_keep]

    # 计算每列的均值和标准差
    means = pseudo_matrix.mean()
    stds = pseudo_matrix.std()
    # 计算变异系数
    cv = (stds / means).abs() * 100
    # 按变异系数从大到小排序
    sorted_columns = cv.sort_values(ascending=False).index
    # 选取变异系数最高的前2000列
    top_2000_columns = sorted_columns[:min(2000,pseudo_matrix.shape[1])]

    if pseudo_matrix.shape[1] > 0:
            #normalized_pseudo_matrix = quantile_transformer.fit_transform(pseudo_matrix)
            #normalized_pseudo_matrix = scaler.fit_transform(normalized_pseudo_matrix)
            #normalized_pseudo_matrix = pd.DataFrame(normalized_pseudo_matrix, columns=pseudo_matrix.columns,index=pseudo_matrix.index)
            #normalized_pseudo_matrix.to_csv(f'/CIMA/Data/eQTL/normal_dis/{celltype}.csv')
            #normalized_pseudo_matrix[top_2000_columns].to_csv(f'/CIMA/Data/eQTL/top2000_normal_dis/{celltype}.csv')
        pseudo_matrix.to_csv(f'/CIMA/Data/dynamic/pseudobulk/pseudobulk/{celltype}.csv')
        pseudo_matrix[top_2000_columns].to_csv(f'/CIMA/Data/dynamic/pseudobulk/top2000_pseudobulk/{celltype}.csv')
    print(f"处理完成 {celltype}", flush=True)


In [65]:
adata_B_use_for_dynamic = adata[adata.obs["celltype_l1"] == "B"]
adata_B_use_for_dynamic = adata_B_use_for_dynamic[adata_B_use_for_dynamic.obs["celltype_l2"] != 'Total_Plasma']
del adata_B_use_for_dynamic.layers

In [67]:
preprocess_RNA_data(adata_use=adata_B_use_for_dynamic,celltype="B")

正在处理 B
处理完成 B


In [64]:
adata_Mono_use_for_dynamic = adata[adata.obs["celltype_l2"] == "Monocyte"]
del adata_Mono_use_for_dynamic.layers

In [68]:
preprocess_RNA_data(adata_use=adata_Mono_use_for_dynamic,celltype="Monocyte")

正在处理 Monocyte
处理完成 Monocyte


In [ ]:
##make_besd

In [ ]:
#解决R语言把“-”换成“."的问题
tss = pd.read_csv('/CIMA/Make_Gene_TSS/tss.bed',sep = ' ')
#为了和R得到的结果相符合
tss['gene_id_map'] = tss['gene_id'].str.replace('-', '.', regex=False)
# 创建字典，映射 gene_id_map 到 gene_id，只包含 B 数据框中出现的列
gene_map_dict = pd.Series(tss.gene_id.values, index=tss.gene_id_map).to_dict()

In [71]:
for cell in ['B','Monocyte']:
    print(f'processing_{cell}')
    #make_bed_file_eQTL
    ## Prepare phenotype data
    norm_expr = pd.read_csv(f'/CIMA/Data/dynamic/pseudobulk/normal_dis/{cell}.csv', index_col=0)
    #解决R语言列名把“-”变成'.'的问题
    filtered_map_dict = {key: gene_map_dict[key] for key in norm_expr.columns}
    # 使用字典重命名 B 数据框的列
    norm_expr.rename(columns=filtered_map_dict, inplace=True)

    norm_expr = norm_expr.T
    norm_expr = norm_expr.reset_index()   # gene x sample

    pheno_file =  pd.merge(tss, norm_expr, right_on='index', left_on='gene_id', how='inner')
    pheno_file = pheno_file.drop(['index'], axis=1).rename(columns={'chr':'#chr'})
    pheno_file['#chr'] = pheno_file['#chr'].str.replace('chr','')    # required phenotype file

    if norm_expr.shape[0] != pheno_file.shape[0]:
        raise ValueError("The number of rows in the two dataframes are not equal.")
    else:
        pheno_file.to_csv(f'/CIMA/Data/dynamic/pseudobulk/bed_file/{cell}.bed', sep='\t', index=False)

processing_B
processing_Monocyte


In [1]:
import os
os.environ["R_HOME"] = "/home/huangzhuoli/mambaforge/envs/tensorqtl/lib/R"
os.environ["R_LIBS_USER"] = "/home/huangzhuoli/mambaforge/envs/tensorqtl/lib/R/library"
import pandas as pd
import tensorqtl

In [80]:
B_cell_result = pd.read_csv('/CIMA/Result/dynamic/pseudobulk/B/all_lead_perm.csv')

tensorqtl.calculate_qvalues(B_cell_result)

B_cell_result = B_cell_result[B_cell_result['qval'] < 0.01]
print(len(B_cell_result))
sig_eQTL = pd.read_csv('/CIMA/Result/20250108_cis_eQTL_studywise_sig.csv',index_col=0)

B_cell_result = B_cell_result[B_cell_result['phenotype_id'].isin(sig_eQTL['phenotype_id'])].reset_index(drop=True)
print(len(B_cell_result))

B_cell_result.to_csv('/CIMA/Result/dynamic/pseudobulk/B/20250122_eGene_use_for_down_stream.csv')

Computing q-values
  * Number of phenotypes tested: 13406
  * Correlation between Beta-approximated and empirical p-values: 0.9999
  * Proportion of significant phenotypes (1-pi0): 0.58
  * QTL phenotypes @ FDR 0.05: 5442
  * min p-value threshold @ FDR 0.05: 0.0484728
4290
4013


In [81]:
Mono_cell_result = pd.read_csv('/CIMA/Result/dynamic/pseudobulk/Monocyte/all_lead_perm.csv')

tensorqtl.calculate_qvalues(Mono_cell_result)

Mono_cell_result = Mono_cell_result[Mono_cell_result['qval'] < 0.01]
print(len(Mono_cell_result))

sig_eQTL = pd.read_csv('/CIMA/Result/20250108_cis_eQTL_studywise_sig.csv',index_col=0)

Mono_cell_result = Mono_cell_result[Mono_cell_result['phenotype_id'].isin(sig_eQTL['phenotype_id'])].reset_index(drop=True)

print(len(Mono_cell_result))

Mono_cell_result.to_csv('/CIMA/Result/dynamic/pseudobulk/Monocyte/20250122_eGene_use_for_down_stream.csv')

Computing q-values
  * Number of phenotypes tested: 12821
  * Correlation between Beta-approximated and empirical p-values: 0.9999
  * Proportion of significant phenotypes (1-pi0): 0.64
  * QTL phenotypes @ FDR 0.05: 5465
  * min p-value threshold @ FDR 0.05: 0.0586476
4206
3931


In [82]:
#make besd for SMR
import pandas as pd
import numpy as np
import os

In [83]:
eQTL_dir = '/CIMA/Result/dynamic/pseudobulk/'
eQTL_temp_dir = '/CIMA/Data/dynamic/for_SMR/result_for_besd/'
eQTL_output_dir = '/CIMA/Data/dynamic/for_SMR/besd/'
smr = '/media/scPBMC1_AnalysisDisk1/huangzhuoli/Script_HPC/software_gaoyue/SMR/smr-1.3.1-linux-x86_64/smr-1.3.1'

In [84]:
CT_list = ['B','Monocyte']

In [ ]:
#update_esi
bim = pd.read_csv('/CIMA/genetics/qc/10.maf01.bim', header=None, sep='\t')
bim.columns = ['chr','variant_id','dis','pos','A1','A2']
bed_df = pd.read_csv('/CIMA/Data/SMR/tss.strand.csv')

In [87]:
for celltype in CT_list:
    print('processing_'+celltype)
    eGene = pd.read_csv(f'/CIMA/Result/dynamic/pseudobulk/{celltype}/20250122_eGene_use_for_down_stream.csv')
    eGene = eGene['phenotype_id'].unique()
    
    parquet_files = [f for f in os.listdir(f'{eQTL_dir}{celltype}') if f.endswith('.parquet')]

    # 初始化一个空的 DataFrame 来存储结果
    eQTL_match = pd.DataFrame()

    # 循环遍历每个.parquet文件
    for file in parquet_files:
        # 读取.parquet文件为 DataFrame
        file_path = os.path.join(f'{eQTL_dir}{celltype}', file)
        eQTL_result = pd.read_parquet(file_path)    
        # 过滤数据并合并
        eQTL_result_match = eQTL_result[eQTL_result['phenotype_id'].isin(eGene)]
        eQTL_match = pd.concat([eQTL_match, eQTL_result_match])
    
    #make besd
    eQTL_match_besd = eQTL_match.loc[:,['variant_id','phenotype_id','slope','slope','pval_nominal','pval_nominal']]
    eQTL_match_besd.columns = ['SNP','gene','beta','t-stat','p-value','FDR']
    eQTL_match_besd ['t-stat'] = 'NA'
    eQTL_match_besd ['FDR'] = 'NA'
    eQTL_match_besd.to_csv(f'{eQTL_temp_dir}/{celltype}_eQTL_for_besd.txt', index=False, sep='\t')

    #linux command
    eqtl = f'{eQTL_temp_dir}/{celltype}_eQTL_for_besd.txt'
    gi = f'{smr} --eqtl-summary {eqtl} --matrix-eqtl-format --make-besd --out {eQTL_output_dir}{celltype}'
    os.system(gi)
    
    print('updating_'+celltype)
    esi = pd.read_csv(f'{eQTL_output_dir}{celltype}.esi', sep='\t', header=None)
    esi.columns = ['chr','variant_id','dis','pos','A1','A2','af']
    esi = esi[['variant_id']]
    esi = pd.merge(esi, bim.loc[:,['chr','variant_id','dis','pos','A1','A2']], left_on='variant_id', right_on='variant_id')

    af_df = eQTL_match.loc[:,['variant_id','af']]
    af_df_dup = af_df.duplicated(subset='variant_id', keep='first')
    af_df = af_df.loc[~af_df_dup]

    esi = pd.merge(esi, af_df, left_on='variant_id', right_on='variant_id')
    esi = esi.loc[:,['chr','variant_id','dis','pos','A1','A2','af']]
    esi.to_csv(f'{eQTL_output_dir}{celltype}_update.esi', sep='\t', header=None, index=False)

    #linux command
    gi = f'{smr} --beqtl-summary {eQTL_output_dir}{celltype} --update-esi {eQTL_output_dir}{celltype}_update.esi'
    os.system(gi)

    #updata_epi
    epi = pd.read_csv(f'{eQTL_output_dir}{celltype}.epi', sep='\t', header=None)
    epi.columns = ['chr','prob','dis','pos','gene_id','strand']
    epi = epi[['prob']]
    epi = pd.merge(epi, bed_df, left_on='prob',right_on='prob')
    epi.loc[:,'dis'] = 0
    epi = epi.loc[:,['chr','prob','dis','pos','gene_id','strand']]
    epi.to_csv(f'{eQTL_output_dir}{celltype}_update.epi', index=False, header=None, sep='\t')

    #linux command
    gi = f'{smr} --beqtl-summary {eQTL_output_dir}{celltype} --update-epi {eQTL_output_dir}{celltype}_update.epi'
    os.system(gi)

processing_B
*******************************************************************
* Summary-data-based Mendelian Randomization (SMR)
* Version 1.3.1
* Build at Sep 21 2022 12:13:19, by GCC 8.3
* (C) 2015 Futao Zhang, Zhihong Zhu and Jian Yang
* The University of Queensland
* MIT License
*******************************************************************
Analysis started: 16:36:42,Wed Jan 22,2025

Options:
--eqtl-summary /CIMA/Data/dynamic/for_SMR/result_for_besd//B_eQTL_for_besd.txt
--matrix-eqtl-format 
--make-besd 
--out /CIMA/Data/dynamic/for_SMR/besd/B

Reading eQTL summary data from /CIMA/Data/dynamic/for_SMR/result_for_besd//B_eQTL_for_besd.txt ...
14643552 rows to be included from /CIMA/Data/dynamic/for_SMR/result_for_besd//B_eQTL_for_besd.txt.

Generating the .epi file...
4013 probes have been saved in the file /CIMA/Data/dynamic/for_SMR/besd/B.epi.

Generating the .esi file...
3708514 SNPs have been saved in the file /CIMA/Data/dynamic/for_SMR/besd/B.esi.

Generating the .besd 